# Importazione 

Selezionare il conda environment **camera_cafe**

Questo script serve per creare gli inverted indexes e vocabulary per i vari casi.   
In particolare si distinguono due casi:
   - Ricerca su **Titolo episodio** e **Trama**

   - Ricerca su **Copione episodio**

Forse sarebbe il caso di utilizzare funzioni di stemming diverse per il titolo ed il copione episodio


## Importazione pacchetti

In [1]:
import pandas as pd # Lavorare su dataframes
import json         # Lavorare con file json
import re           # Lavorare con regular expression
import numpy as np  # Lavorare con matrici 

# Pacchetti per calcolare i valori  tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer # TF IDF
from sklearn.metrics.pairwise import cosine_similarity      # Cosine similarity


from collections import Counter # Dizionario speciale
from functools import reduce

import math

# Pacchetti per lavorare sul testo 
import spacy                              # Per lemmatizzazione
nlp = spacy.load('it_core_news_sm') # Caricare lemmatizzazione italiana

from nltk.stem import SnowballStemmer # Stemming
stemmer_snowball = SnowballStemmer('italian') # Definire lo stemmer per la lingua italiana

import unicodedata # Questo pacchetto serve per normalizzare le stringhe.  
                   # Ad esempio le due stringhe "l'amore è cieco"=="l'amore è cieco"
                   # risultavano diverse, alcuni caratteri nascosti le facevano sembrare diverse in termini di significato
                   # per un umano fossero uguali (si possono capire le differenze tramite il seguente for loop)
                   # for a, b in zip(r"l'amore è cieco", r"l'amore è cieco"):
                   #    print(a, b, a == b)  
                   # Tramite il pacchetto unicodedata si possono normalizzare le stringhe e renderle uguali


# Importare le funzioni
from  lemmatizzatore import lemmatizzazione

## Importazione dati

Importare il dataset creato nel file **01_PuliziaDati** con tutte le informazioni necessarie, titolo episodio, trama, trascrizione della puntata, ecc....

In [ ]:
# Importare il dataset contenente tutti i dati per ogni episodio
df=pd.read_csv(filepath_or_buffer="/Dati/dati_puliti.csv",
               index_col="Unnamed: 0" # Indicare la colonna che fa da index
               )
df.head(2)

,season,episodio,titolo,trama,trascrizione_puntata,guest_star,prima_visione,title_trama,-,Adriana De Guilmi,...,Valeria,Brad,Jonathan,Signora Bollini,Mamma di Silvano,Wanda,Vittorio,Carminati,Caterina,Roberto
0,1,0,Sketch Promo,Sketch Promo la prima puntata di camera cafe,"Buongiorno signori. Buongiorno direttore, il ...",NaN,NaN,Sketch Promo Sketch Promo la prima puntata d...,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,Passaportafogli,Paolo dimentica il suo portafogli su un taxi e...,"Musica Ma che schifo, cosa fai? Mangi i corne...",NaN,6 ottobre 2003,Passaportafogli Paolo dimentica il suo portafo...,0,0,...,0,0,0,0,0,0,0,0,0,0


# Funzioni

Definire le funzioni per la pulizia del testo.  
   - Lemmatizzazione
   - Standardizzazione 
   - Rimozione punteggiatura

## Cleaning generico del testo
Rimuovere spazi e punteggiatura

## Lemmatizzazione


Lemmatizzazione del testo:
   - Portare tutti i verbi all'infinito 

   - Tutti i nomi al maschile

   - articoli ricodificati nella forma maschile

   - Preposizioni ricodificati 

Altre modifiche al testo:

   - Rimozione congiunzioni 

   - Rimozione preposizioni

   - Portare tutto ciò che non è un nome in minuscolo 

   - Ottenere una lista dei personaggi presenti

Nota: Questa funzione dovrà essere applicata anche alle query degli utenti


# Creazione index e inverted index

## Titolo episodio e trama

In [3]:
# Applicare la funzione di lemmatizzazione
df['list_words'], output_inutile = zip(*df['title_trama'].apply(lambda row: lemmatizzazione(row) if type(row) != float else ([], [])))

# Vocabulary
vocabulary_title_trama =Counter(reduce(lambda x, y: x + y, df.list_words))

# Word dictionary
word_dict_title_trama = {}
item_id = 1
for word in vocabulary_title_trama.keys():
    word_dict_title_trama[word] = item_id
    item_id += 1

# Inverted indexes
inverted_idx_title_trama = {}
for word, item_id in word_dict_title_trama.items():
    inverted_idx_title_trama[item_id] = list(df[df.list_words.apply(lambda row: word in row)].index)

# Tf-IDF
tfidf_title_trama = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text:  text)
result_title_trama = tfidf_title_trama.fit_transform(df.list_words)
result_dense_title_trama = result_title_trama.todense()
results_dense_list_title_trama = result_dense_title_trama.tolist()

/opt/anaconda3/envs/camera_cafe/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Tutto insieme

In [4]:
# Applicare la funzione di lemmatizzazione
df['list_words_all'], output_inutile = zip(*df['title_trama_trascrizione'].apply(lambda row: lemmatizzazione(row) if type(row) != float else ([], [])))

# Vocabulary
vocabulary_all =Counter(reduce(lambda x, y: x + y, df.list_words_all))

# Word dictionary
word_dict_all = {}
item_id = 1
for word in vocabulary_all.keys():
    word_dict_all[word] = item_id
    item_id += 1

# Inverted indexes
inverted_idx_all = {}
for word, item_id in word_dict_all.items():
    inverted_idx_all[item_id] = list(df[df.list_words_all.apply(lambda row: word in row)].index)

# Tf-IDF
tfidf_all = TfidfVectorizer(input='content', lowercase=False, tokenizer=lambda text:  text)
result_all = tfidf_all.fit_transform(df.list_words_all)
result_dense_all = result_all.todense()
results_dense_list_all = result_dense_all.tolist()

/opt/anaconda3/envs/camera_cafe/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## Esportazione td_idf index, vocabulary, word dictionary, inverted index

In [ ]:
df.to_csv("/Dati/dati_puliti_aggiornati.csv") #esportazione dati

# Salva il DataFrame in formato Parquet con compressione
df.to_parquet("/Dati/dati_puliti_aggiornati.parquet", compression='brotli')

In [ ]:
DF3_title_trama=pd.DataFrame(results_dense_list_title_trama, index=df.index, columns=tfidf_title_trama.get_feature_names_out())
DF3_title_trama.to_csv("/Dati/tfidf_index_title_trama.csv")

# Salva il DataFrame in formato Parquet con compressione
DF3_title_trama.to_parquet("/Dati/tfidf_index_title_trama.parquet", compression='brotli')

with open("/Dati/vocabulary_title_trama.json", 'w') as v:
    json.dump(vocabulary_title_trama, v, indent=4)
    
with open("/Dati/word_dict_title_trama.json", 'w') as wd:
    json.dump(word_dict_title_trama, wd, indent=4)

with open("/Dati/inverted_idx_title_trama.json", 'w') as inv_idx:
    json.dump(inverted_idx_title_trama, inv_idx, indent=4)

In [ ]:
DF3_all=pd.DataFrame(results_dense_list_all, index=df.index, columns=tfidf_all.get_feature_names_out())
DF3_all.to_csv("/Dati/tfidf_index_all.csv")

# Salva il DataFrame in formato Parquet con compressione
DF3_all.to_parquet("/Dati/tfidf_index_all.parquet", compression='brotli')



with open("/Dati/vocabulary_all.json", 'w') as v:
    json.dump(vocabulary_all, v, indent=4)
    
with open("/Dati/word_dict_all.json", 'w') as wd:
    json.dump(word_dict_all, wd, indent=4)

with open("/Dati/inverted_idx_all.json", 'w') as inv_idx:
    json.dump(inverted_idx_all, inv_idx, indent=4)